In [8]:
from tqdm import tqdm
import pandas as pd 
import os
from snorkel.labeling import PandasLFApplier,LFAnalysis,LabelingFunction
from snorkel.labeling.model.label_model import LabelModel
from snorkel.labeling.apply.dask import PandasParallelLFApplier
import re
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import pytrends
from pytrends.request import TrendReq
import time
import datetime
from datetime import datetime, date, time
from nltk import ngrams

In [15]:
gold=pd.read_csv('hypercoagulable_sentences.csv')
gold=gold.dropna()

In [21]:
gold.head()

,Unnamed: 0,sid,ncord_uid,sentence
0,0,0,0,… 2017; Accepted: June 2017 CASE REPORT Pneuma...
1,1,1,0,[6] Patients with hypercoagulable states are p...
2,2,2,1,… relates to materials and methods for using l...
3,3,3,2,Ehrlichia canis infection in dogs can cause t...
4,4,4,2,It is unknown why some dogs show signs of ble...


In [106]:

ab=[]
ncord=[]
for i in tqdm(gold.ncord_uid.drop_duplicates()):
    temp=gold[gold.ncord_uid==i]
    tempsent=temp.sentence.values
    tempncord=temp.ncord_uid.drop_duplicates().values[0]
    init=''
    for j in tempsent:
        init=init+j
    ab.append(init)
    ncord.append(tempncord)
goldab=pd.DataFrame(ncord,columns=['ncord_uid'])
goldab['abstract']=ab
goldab=goldab.drop_duplicates()

100%|██████████| 8766/8766 [00:13<00:00, 638.06it/s]


In [ ]:
'health services','basic science','diagnostic','prevention','screening','device'

In [107]:
triallist=['single group assignment','parallel assignment','crossover assignment','sequential sssignment','factorial assignment'
           ,'cohort','case only','case-control','case control','ecologic or Community','family based'  
         ,'open label','double blind','single blind','triple blind','quadruple blind',
         'double-blind','single-blind','triple-blind','quadruple-blind'
          ,'treatment','supportive care'
        ,'randomized','non-Randomized','non-Randomized'
         ]

keywordslist=['coagulation','D-dimer','thromboelastometry','anticoagulants','venous thromboembolism','clot formation time'
,'Maximum Clot Firmness', 'PT', 'aPTT', 'thrombin time','platelet','fibrinogen','antiphospholipid antibody',
'heparin induced thrombocytopenia','prothrombin time','platelet count','factor v','factor ii','protein c','protein s',
'antithrombin iii','leiden/apc','c-reactive protein','coagulopathy','disseminated intravascular coagulation',
'thrombotic microangiopathy','thrombotic manifestations','sic score','venous thromboembolic','arterial thrombosis'
,'thrombosis','heparin','tissue plasminogen activator','anticoagulants','prophylactic','anticoagulation'
             ]
viruslist=['covid-19','covid 19','sars cov 2','sars-cov-2','sars like cov']
mustlist=['hypercoagulable']

In [141]:
gold.head()

,Unnamed: 0,sid,ncord_uid,sentence,label,prob_snokel
108166,108166,108166,7858,COVID-19 is an infectious disease caused by th...,1,0.999999
17719,17719,17719,1232,"In all cases, medical treatment including ant...",1,0.999998
92919,92919,92919,6768,We examine current diagnostic and treatment o...,1,0.999998
63337,63337,63337,4318,METHODS Electronic databases including MEDLIN...,1,0.999998
120359,120359,120359,8393,12 bleeding episodes were observed; 8 occurre...,1,0.999998


In [133]:
def loop_labing(keywordslist,viruslist,triallist,mustlist,maxngramnu):
    
    
    def keyword_lookup(x,keywords,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.abstract.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_keyword_lf(keywords,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_{name}",
            f=keyword_lookup,
            resources=dict(keywords=keywords,maxngram=maxngram,label=label),)
    
    def keyword_lookup_b(x,keywords,must,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.abstract.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords) and any(word1 in wordlist  for word1 in must):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_keyword_b_lf(keywords,must,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_b_{name}",
            f=keyword_lookup_b,
            resources=dict(keywords=keywords,must=must,maxngram=maxngram,label=label),)
    
    def trial_lookup(x,trial,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.abstract.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in trial):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_trial_lf(trial,maxngram,label=None):
        return LabelingFunction(
            name=f"trial",
            f=trial_lookup,
            resources=dict(trial=trial,maxngram=maxngram,label=label),)
    
    def not_lookup(x,keywords,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.abstract.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold=threshold+1
        if threshold<1 and threshold1>0:
            return label
        return Norelevent

    def make_not_lf(keywords,name,maxngram,label=None):
        return LabelingFunction(
            name=f"not_{name}",
            f=not_lookup_b,
            resources=dict(keywords=keywords,maxngram=maxngram,label=label),)  
    
    def not_lookup_b(x,keywords,virus,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.abstract.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in virus):
                    threshold=threshold+1
        threshold1=0
        for i in range(maxngram):
            gramab=ngrams(x.abstract.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold1=threshold1+1
        if threshold<1 and threshold1>0:
            return label
        return Norelevent

    def make_not_b_lf(keywords,virus,name,maxngram,label=None):
        return LabelingFunction(
            name=f"not_b_{name}",
            f=not_lookup_b,
            resources=dict(keywords=keywords,virus=virus,maxngram=maxngram,label=label),)   
    
    def number_lookup(x,trial,label):
        threshold=0
        sentlist=x.abstract.split()
        trialindex=[i for i, e in enumerate(sentlist) if e in trial]
        nuindex=[i for i, e in enumerate(sentlist) if str(e).isdigit()]
        for i in trialindex:
            for j in nuindex:
                if i<j:
                    threshold+=1
        if threshold>0:
            return label
        return Norelevent

    def make_number_lf(trial,label=None):
        return LabelingFunction(
            name=f"number",
            f=number_lookup,
            resources=dict(trial=trial,label=label),)

   
    
    Norelevent = -1
    
    keywordfu=make_keyword_lf(keywords=keywordslist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfu=make_keyword_lf(keywords=viruslist,maxngram=maxngramnu,name='virus',label=1)
    keywordfub=make_keyword_b_lf(keywords=keywordslist,must=mustlist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfub=make_keyword_b_lf(keywords=viruslist,must=mustlist,maxngram=maxngramnu,name='virus',label=1)
    trialfu=make_trial_lf(trial=triallist,maxngram=maxngramnu,label=1)
    notfu1=make_not_b_lf(keywords=mustlist,virus=keywordslist,maxngram=maxngramnu,name='notkeyword',label=0)
    notfu2=make_not_b_lf(keywords=keywordslist,virus=triallist,maxngram=maxngramnu,name='nottrial',label=0)
    numberfu=make_number_lf(trial=triallist,label=1)
    
    allweaklabf=[]

    allweaklabf.append(keywordfu)
    allweaklabf.append(virusfu)
    allweaklabf.append(keywordfub)
    allweaklabf.append(virusfub)
    allweaklabf.append(trialfu)
    allweaklabf.append(notfu1)
    allweaklabf.append(notfu2)
    allweaklabf.append(numberfu)

    
    return allweaklabf


In [134]:
cardinalitynu=2
allweaklabf=loop_labing(keywordslist,viruslist,triallist,mustlist,3)
applier = PandasLFApplier(lfs=allweaklabf)
all_train_l = applier.apply(df=gold)
#applier = PandasParallelLFApplier(lfs=allweaklabf)
#all_train_l = applier.apply(df=allframe1,n_parallel=40)
report=LFAnalysis(L=all_train_l, lfs=allweaklabf).lf_summary()
print(report)
label_model = LabelModel(cardinality=cardinalitynu,verbose=True)
label_model.fit(all_train_l)
predt=label_model.predict(all_train_l)


100%|██████████| 8766/8766 [02:43<00:00, 53.63it/s] 


                   j Polarity  Coverage  Overlaps  Conflicts
keyword_keyword    0      [1]  0.236140  0.236140   0.130504
keyword_virus      1      [1]  0.060461  0.048597   0.004905
keyword_b_keyword  2       []  0.000000  0.000000   0.000000
keyword_b_virus    3      [1]  0.000570  0.000570   0.000570
trial              4      [1]  0.621948  0.497718   0.000684
not_b_notkeyword   5      [0]  0.001141  0.000799   0.000799
not_b_nottrial     6      [0]  0.130504  0.130504   0.130504
number             7      [1]  0.450605  0.450605   0.000228


In [135]:
goldab['label']=predt
prob=label_model.predict_proba(all_train_l)[:,1]
goldab['prob_snokel']=prob
goldab1=goldab[goldab.label==1]
goldab1=goldab1.drop_duplicates()
goldab1=goldab1.sort_values(by='prob_snokel',ascending=False)

In [140]:
goldab.to_pickle('./round_2ab.pkl')

In [145]:
def loop_labing(keywordslist,viruslist,triallist,mustlist,maxngramnu):
    
    
    def keyword_lookup(x,keywords,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_keyword_lf(keywords,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_{name}",
            f=keyword_lookup,
            resources=dict(keywords=keywords,maxngram=maxngram,label=label),)
    
    def keyword_lookup_b(x,keywords,must,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords) and any(word1 in wordlist  for word1 in must):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_keyword_b_lf(keywords,must,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_b_{name}",
            f=keyword_lookup_b,
            resources=dict(keywords=keywords,must=must,maxngram=maxngram,label=label),)
    
    def trial_lookup(x,trial,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in trial):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_trial_lf(trial,maxngram,label=None):
        return LabelingFunction(
            name=f"trial",
            f=trial_lookup,
            resources=dict(trial=trial,maxngram=maxngram,label=label),)
    
    def not_lookup(x,keywords,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold=threshold+1
        if threshold<1 and threshold1>0:
            return label
        return Norelevent

    def make_not_lf(keywords,name,maxngram,label=None):
        return LabelingFunction(
            name=f"not_{name}",
            f=not_lookup_b,
            resources=dict(keywords=keywords,maxngram=maxngram,label=label),)  
    
    def not_lookup_b(x,keywords,virus,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in virus):
                    threshold=threshold+1
        threshold1=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold1=threshold1+1
        if threshold<1 and threshold1>0:
            return label
        return Norelevent

    def make_not_b_lf(keywords,virus,name,maxngram,label=None):
        return LabelingFunction(
            name=f"not_b_{name}",
            f=not_lookup_b,
            resources=dict(keywords=keywords,virus=virus,maxngram=maxngram,label=label),)   
    
    def number_lookup(x,trial,label):
        threshold=0
        sentlist=x.sentence.split()
        trialindex=[i for i, e in enumerate(sentlist) if e in trial]
        nuindex=[i for i, e in enumerate(sentlist) if str(e).isdigit()]
        for i in trialindex:
            for j in nuindex:
                if i<j:
                    threshold+=1
        if threshold>0:
            return label
        return Norelevent

    def make_number_lf(trial,label=None):
        return LabelingFunction(
            name=f"number",
            f=number_lookup,
            resources=dict(trial=trial,label=label),)

   
    
    Norelevent = -1
    
    keywordfu=make_keyword_lf(keywords=keywordslist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfu=make_keyword_lf(keywords=viruslist,maxngram=maxngramnu,name='virus',label=1)
    keywordfub=make_keyword_b_lf(keywords=keywordslist,must=mustlist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfub=make_keyword_b_lf(keywords=viruslist,must=mustlist,maxngram=maxngramnu,name='virus',label=1)
    trialfu=make_trial_lf(trial=triallist,maxngram=maxngramnu,label=1)
    notfu1=make_not_b_lf(keywords=mustlist,virus=keywordslist,maxngram=maxngramnu,name='notkeyword',label=0)
    notfu2=make_not_b_lf(keywords=keywordslist,virus=triallist,maxngram=maxngramnu,name='nottrial',label=0)
    numberfu=make_number_lf(trial=triallist,label=1)
    
    allweaklabf=[]

    allweaklabf.append(keywordfu)
    allweaklabf.append(virusfu)
    allweaklabf.append(keywordfub)
    allweaklabf.append(virusfub)
    allweaklabf.append(trialfu)
    allweaklabf.append(notfu1)
    allweaklabf.append(notfu2)
    allweaklabf.append(numberfu)

    
    return allweaklabf


In [146]:
cardinalitynu=2
allweaklabf=loop_labing(keywordslist,viruslist,triallist,mustlist,3)
applier = PandasLFApplier(lfs=allweaklabf)
all_train_l = applier.apply(df=gold)
#applier = PandasParallelLFApplier(lfs=allweaklabf)
#all_train_l = applier.apply(df=allframe1,n_parallel=40)
report=LFAnalysis(L=all_train_l, lfs=allweaklabf).lf_summary()
print(report)
label_model = LabelModel(cardinality=cardinalitynu,verbose=True)
label_model.fit(all_train_l)
predt=label_model.predict(all_train_l)

100%|██████████| 119721/119721 [03:38<00:00, 547.47it/s] 


                   j Polarity  Coverage  Overlaps  Conflicts
keyword_keyword    0      [1]  0.028767  0.028767   0.024549
keyword_virus      1      [1]  0.014484  0.003174   0.001328
keyword_b_keyword  2       []  0.000000  0.000000   0.000000
keyword_b_virus    3      [1]  0.000042  0.000042   0.000042
trial              4      [1]  0.084321  0.014935   0.000000
not_b_notkeyword   5      [0]  0.000326  0.000142   0.000142
not_b_nottrial     6      [0]  0.024549  0.024549   0.024549
number             7      [1]  0.009965  0.009965   0.000000


In [147]:
gold['label']=predt
prob=label_model.predict_proba(all_train_l)[:,1]
gold['prob_snokel']=prob
gold1=gold[gold.label==1]
gold1=gold1.drop_duplicates()
gold1=gold1.sort_values(by='prob_snokel',ascending=False)

In [148]:
gold.to_pickle('./round_2.pkl')

In [137]:
goldab[goldab.abstract.str.contains('hypercoagulable')]

,ncord_uid,abstract,label,prob_snokel
6287,12,Background and Purpose Adjunctive treatments l...,1,0.985477
4209,5362,Background and Purpose Adjunctive treatments l...,1,0.985477
61,3167,Despite treatment with prophylactic enoxapari...,1,0.929399
3564,7896,Medical records information including demogra...,1,0.929399
43,7068,Despite treatment with prophylactic enoxapari...,1,0.929399
66,1236,Despite treatment with prophylactic enoxapari...,1,0.929399
29,2169,Despite treatment with prophylactic enoxapari...,1,0.929399
33,7800,Despite treatment with prophylactic enoxapari...,1,0.929399
10,13,Despite treatment with prophylactic enoxapari...,1,0.929399
687,16,Operative treatment was performed in 17 (85%)...,1,0.929399


In [41]:
gold1=gold1.sort_values(by='prob_snokel',ascending=False)

In [59]:
threshold=0
for word in ngrams('4 years and mean BMI was 42'.split(), 3):
    wordlist=list(word)
    if any(word1 in wordlist  for word1 in keywords):
        threshold=threshold+1

In [60]:
threshold

0

In [43]:
gold1.head().sentence.values

array(['… 2017; Accepted: June 2017 CASE REPORT Pneumatosis intestinalis, to treat or not to treat … with prodromal abdominal angina',
       '4 years and mean BMI was 42',
       'The major impact produced by the severe acute respiratory syndrome coronavirus 2 (SARS‑CoV‑2) focused many researchers attention to find treatments that can suppress transmission or ameliorate the disease',
       ' Here, we describe the antiviral effects of two molecules that alter polyamine levels: difluoromethylornithine (DFMO; also called eflornithine), which is a suicide inhibitor of ornithine decarboxylase 1 (ODC1), and diethylnorspermine (DENSpm), an activator of spermidine/spermine N1-acetyltransferase (SAT1)',
       ' High and low mesh compactions were also achieved for 2 real patients by using the dynamic push-pull technique'],
      dtype=object)

In [ ]:
tokenlist=gold.text.values[1].splitlines()
tokenlist=[i for i in tokenlist if i]
paracat=[]
paracot=[]
for i in range(len(tokenlist)):
    temp=tokenlist[i].replace(':::','')
    nuword=len(word_tokenize(temp))
    if nuword<=10:
        paracat.append(temp)
    else:
        paracot.append(temp)
print(len(paracat))
print(len(paracot))

In [ ]:
paracat

In [ ]:
word_tokenize('F(ab´)2 mechanisms ::: Immunomodulation')

In [ ]:
paracat

In [ ]:
tokenlist

In [ ]:
paracat

In [ ]:
paracot